In [1]:
'''
Module to create nine centrality outputs (betweenness, closeness,
degree, efficiency, eigenvector, harmonic, katz, straightness, and current flow)

Created on Dec 29, 2020

@author: 
'''
from PyQt5.QtWidgets import QFileDialog
from PyQt5.QtWidgets import QInputDialog
from PyQt5.QtWidgets import QApplication

import sys, os
import loadApplyModel
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

from networkAnalysis import NetworkAnalysis
import csv

'''
Class extends NetworkAnalysis
'''
class NetworkOutput(NetworkAnalysis):
    
    '''
    Method to output efficiency centrality to .csv.
    @param results-- the results to print out
    @param loc-- the directory location to printout the file to
    @param fileType-- the file for output results
    '''
    def printResults(self,results,loc,fileType):
        
        fileN=self.getOutuptPath(loc,fileType)
        
        fieldnames = ['id','x','y','value']
        
        with open(fileN, 'w') as csvf:
            writer = csv.DictWriter(csvf, fieldnames=fieldnames)

            writer.writeheader()
            
            i=0
            for ie in results:
            
                value=results[ie]
            
                writer.writerow({'id':i,'x':str(ie[0]),'y':str(ie[1]), 'value' :str(value)})
           
                i+=1
    '''
     Method to print nine node centrality values to csv file.

     @param loc-- the folder location to print out to
     @param fileType-- the name of the file to print results to
     @param result1-- efficiency centrality value
     @param result2-- straightness centrality value
    '''
    def printNodeCentrality(self,loc,fileType,result1,result2):
       
        fileN=self.getOutuptPath(loc,fileType)
     
        fieldnames = ['id','x','y','betweenness','closeness','degree','efficiency',
                 'straightness','katz','eigenvector','current flow','harmonic']
        
        with open(fileN, 'w') as csvf:
            writer = csv.DictWriter(csvf, fieldnames=fieldnames)

            writer.writeheader()
            
            i=0
            for ie in self.bet_cen.keys():
            
                value1=self.bet_cen[ie]
                value2=self.clo_cen[ie]
                value3=self.deg_cen[ie]
                value4=result1[ie]
                value5=result2[ie]
                value6=self.kcentral[ie]
                value7=self.eVector[ie]
                value8=self.cflow[ie]
                value9=self.harmonic[ie]
                #value10=self.dispersion[ie]
            
                writer.writerow({'id':i,'x':str(ie[0]),'y':str(ie[1]),
                             'betweenness':str(value1),'closeness':str(value2),
                             'degree':str(value3),'efficiency':str(value4),
                             'straightness':str(value5),'katz':str(value6),
                             'eigenvector':str(value7),'current flow':str(value8),
                             'harmonic':str(value9)})
           
                i+=1
    '''
    Method to print betweenness centrality for edges.
    @param loc-- the folder location to print out to
    @param fileType-- the name of the file to print results to
    @param edges-- edges to get edge values
    '''
    def printEdgeCentrality(self,loc, fileType, edges):
    
        filename=self.getOutuptPath(loc,fileType)
        
        fieldnames = ['id','x','y','value']
        
        with open(filename, 'w') as csvf:
            writer = csv.DictWriter(csvf, fieldnames=fieldnames)

            writer.writeheader()
            
            i=0
            for ie in edges:
            
                value=edges[ie]
            
                n1=ie[0]
                n2=ie[1]
            
                n1x=n1[0]
                n1y=n1[1]
                n2x=n2[0]
                n2y=n2[1]
            
                writer.writerow({'id':i,'x':str(n1x),'y':str(n1y), 'value' :str(value)})
                writer.writerow({'id':i,'x':str(n2x),'y':str(n2y), 'value' :str(value)})
           
                i+=1
    '''
    Method to convert data to shapefile linear data.
    @param loc- the location of the file.
    @param fileType- the name of the file to output.
    '''
    def convertToLine(self,loc,fileType):
        data=self.getOutuptPath(loc,fileType)
        df = pd.read_csv(data)

        #Convert string
        #Create XY column
        df['XY'] = list(zip(df['x'],df['y']))

        #Group by ID. Any aggfunc is possible, python build-in or own. Also possible to have multiple funcs per field.
        aggfuncs = {'XY':list,'value':'first'}
        df2 = df.groupby('id').agg(aggfuncs)

        #Create geodataframe
        geometry = [LineString([Point(p) for p in row]) for row in df2['XY']]
        crs = {'init':'epsg:4326'}
        gdf = gpd.GeoDataFrame(df2, crs=crs, geometry=geometry)

        #Export to file
        gdf.reset_index(inplace=True) #To keep ID column
        del gdf['XY']
    
    #   del gdf['value']
        path_output=self.getOutuptPath(loc,'betweeness_road_data.shp')
        gdf.to_file(path_output, driver="ESRI Shapefile")   
    
    '''
    Method to output edge centrality to .shp file.
    @param loc- the location of the file
    @param fileType- the name of the file to output
    '''
    def centralityEdges(self,loc,fileType):
        data=self.getOutuptPath(loc,fileType)
        df = pd.read_csv(data)

        #Convert string/text/object time to datetime time
        #Create XY column
        df['XY'] = list(zip(df['x'],df['y']))


        #Group by ID. Any aggfunc is possible, python build-in or own. Also possible to have multiple funcs per field.
        aggfuncs = {'XY':list,'betweenness':'first','closeness':'first','degree':'first',
                    'katz':'first','current flow':'first','harmonic':'first','straightness':'first',
                    'efficiency':'first','eigenvector':'first'}
        
        df2 = df.groupby('id').agg(aggfuncs)

        #Create geodataframe
        geometry = [LineString([Point(p) for p in row]) for row in df2['XY']]
        crs = {'init':'epsg:4326'}
        gdf = gpd.GeoDataFrame(df2, crs=crs, geometry=geometry)

        #Export to file
        gdf.reset_index(inplace=True) #To keep ID column
        del gdf['XY']
    
    #   del gdf['value']
        path_output=self.getOutuptPath(loc,'edge_centrality.shp')
        gdf.to_file(path_output, driver="ESRI Shapefile") 

    '''
    Method to print out to .csv file multiple edge centralities.
    @param loc- the location for the file path
    @param fileType- the name of the file to output
    @param edgz- the edge data
    @param G- the graph
    '''
    def printMultipleEdgeCentralities(self,loc, fileType, edgz, G):
       
        filename=self.getOutuptPath(loc,fileType)
        
        
        fieldnames = ['id','x','y','betweenness','closeness','degree','katz',
                      'current flow','harmonic','eigenvector','efficiency','straightness']
        
        with open(filename, 'w') as csvf:
            writer = csv.DictWriter(csvf, fieldnames=fieldnames)

            writer.writeheader()
            
            i=0
            
            betweeness=self.bet_c
            closeness=edgz['closeness']
            degree=edgz['degree']
            katz=edgz['katz']
            cflow=edgz['current flow']
            harmonic=edgz['harmonic']
            egv=edgz['eigenvector']
            efficiency=edgz['efficiency']
            straightness=edgz['straightness']
            
            for e in G.edges():
                bt=0
                if e in betweeness:
                    bt=betweeness[e]
                
                cv=0
                if str(e) in closeness:
                    cv=closeness[str(e)]
                
                dg=0
                if str(e) in degree:
                    dg=degree[str(e)]
                
                kz=0
                if str(e) in katz:
                    kz=katz[str(e)]
                    
                cf=0
                if str(e) in cflow:
                    cf=cflow[str(e)]
                
                hm=0
                if str(e) in harmonic:
                    hm=harmonic[str(e)]
                
                ev=0
                if str(e) in egv:
                    ev=egv[str(e)]
                
                ds=0
                if str(e) in efficiency:
                    ds=efficiency[str(e)]
                    
                st=0
                if str(e) in straightness:
                    st=straightness[str(e)]

            
                n1=e[0]
                n2=e[1]
                
                n1x=n1[0]
                n1y=n1[1]
                n2x=n2[0]
                n2y=n2[1]
            
                writer.writerow({'id':i,'x':str(n1x),'y':str(n1y), 'betweenness':str(bt),'closeness':str(cv),'degree':str(dg)
                                ,'katz':str(kz),'current flow':str(cf),'harmonic':str(hm),'eigenvector':str(ev),'efficiency':str(ds),'straightness':
                str(st)})
                writer.writerow({'id':i,'x':str(n2x),'y':str(n2y), 'betweenness':str(bt),'closeness':str(cv),'degree':str(dg)
                                 ,'katz':str(kz),'current flow':str(cf),'harmonic':str(hm),'eigenvector':str(ev),'efficiency':str(ds),'straightness':
                str(st)})
           
                i+=1
    '''
    Method to call and run the analysis.
    '''        
    def run(self):
        
        app = QApplication(sys.argv)
    
        qid = QFileDialog()

        #fileName = "Enter the file to analyise here."
        filename = QFileDialog.getOpenFileName()


        #outputFolder = "Enter the output folder location here."
        #mode = QLineEdit.Normal
        #text, ok = QInputDialog.getText(qid,outputFolder,filename, mode)
        # text2 = QInputDialog.getText(qid,filename[0], outputFolder, mode)
        paths=[]
        pn=os.getcwd()
        pn=pn.split("src")[0]
        path=os.path.join(pn,'output')
        paths.append(path)
        paths.append(pn)
        self.text2 = QInputDialog.getItem(qid,"Folder Dialog", "Select Folder", paths, 0, False)
  
        G=loadApplyModel.load(filename)
        res=self.runGlobalEfficiency(G)
    
        self.centralityMeasures(G)
    
        self.efficiencyCentrality(G)
        self.straightnessCentrality(G)
    
        #do centrality outputs to .csv files
        self.printGlobalEfficiency(res,self.text2,'globalEfficiency.csv')
        self.printResults(self.efficiency,self.text2,"efficiencyCentrality.csv")
        self.printResults(self.straightness,self.text2,"straightnessCentrality.csv")
    
        self.printNodeCentrality(self.text2,'nodeCentrality.csv',self.efficiency,self.straightness)
        self.printEdgeCentrality(self.text2,'edgeBetweenessCentrality.csv',self.bet_c)
    
        #convert betweeness edge values to .shp file
        self.convertToLine(self.text2,'edgeBetweenessCentrality.csv')
    
        edgz=self.doEdgeCentralities(G)
        self.printMultipleEdgeCentralities(self.text2,'edgeCentrality.csv',edgz, G)
        self.centralityEdges(self.text2,'edgeCentrality.csv')
            
if __name__ == '__main__':
    nt=NetworkOutput()
    nt.run()

/home/maltaweel/anaconda3/lib/python3.8/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


(49.94139109722019, -164.604167052095):(60.67762381090223, -185.23707839940425)
(49.94139109722019, -164.604167052095):(69.8174961380252, -145.36922669002564)
(49.94139109722019, -164.604167052095):(83.86250291177613, -170.78416675210622)
(49.94139109722019, -164.604167052095):(35.68613075411612, -196.6789667868485)
(49.94139109722019, -164.604167052095):(106.74627968666468, -157.2345620827643)
(49.94139109722019, -164.604167052095):(61.279828462873056, -239.51077265826814)
(49.94139109722019, -164.604167052095):(84.76580988973231, -226.26227031491163)
(49.94139109722019, -164.604167052095):(108.55289364257695, -213.3148702975405)
(49.94139109722019, -164.604167052095):(132.64107972140692, -200.066367954184)
(49.94139109722019, -164.604167052095):(157.63257277819315, -186.44148770334562)
(49.94139109722019, -164.604167052095):(184.43067979089153, -171.6874737300622)
(49.94139109722019, -164.604167052095):(162.14520987595813, -128.05778036416322)
(49.94139109722019, -164.604167052095):(

(60.67762381090223, -185.23707839940425):(255.79193104942544, -245.1564412704937)
(60.67762381090223, -185.23707839940425):(281.084526432197, -229.8002226452396)
(60.67762381090223, -185.23707839940425):(304.8716101850416, -216.55172030188308)
(60.67762381090223, -185.23707839940425):(329.56200091584236, -202.39991098057044)
(60.67762381090223, -185.23707839940425):(357.2634149064969, -187.94699933327243)
(60.67762381090223, -185.23707839940425):(92.59447036535204, -338.7992646519456)
(60.67762381090223, -185.23707839940425):(370.15373505832883, -180.93277541647447)
(60.67762381090223, -185.23707839940425):(382.2545131726996, -172.7717855253872)
(60.67762381090223, -185.23707839940425):(113.67163318432833, -386.9756368096056)
(60.67762381090223, -185.23707839940425):(136.25430763323146, -374.3293391182198)
(60.67762381090223, -185.23707839940425):(160.94469836403223, -360.1775297969072)
(60.67762381090223, -185.23707839940425):(185.33398676884764, -346.62792512756533)
(60.6776238109022

(69.8174961380252, -145.36922669002564):(311.1943765254287, -349.33699862702247)
(69.8174961380252, -145.36922669002564):(291.7922244527227, -361.1601850463277)
(69.8174961380252, -145.36922669002564):(271.10530464363035, -325.82541649032936)
(69.8174961380252, -145.36922669002564):(308.664546978291, -394.8682290101066)
(69.8174961380252, -145.36922669002564):(284.0784562760856, -408.66913862434865)
(69.8174961380252, -145.36922669002564):(257.8996473313231, -362.8874507307754)
(69.8174961380252, -145.36922669002564):(233.81146125249305, -376.7381577261027)
(69.8174961380252, -145.36922669002564):(260.3084659392061, -422.50571127587966)
(69.8174961380252, -145.36922669002564):(310.79986998969645, -453.38246524423766)
(69.8174961380252, -145.36922669002564):(334.73542362884865, -440.006714681182)
(69.8174961380252, -145.36922669002564):(357.26300352452125, -424.5190035029071)
(69.8174961380252, -145.36922669002564):(286.8643163505443, -470.2781501659921)
(69.8174961380252, -145.36922669

(83.86250291177613, -170.78416675210622):(438.2214937745948, -556.164548518244)
(83.86250291177613, -170.78416675210622):(460.04508679852773, -601.2197083095894)
(83.86250291177613, -170.78416675210622):(413.58195326370287, -570.9482728247792)
(83.86250291177613, -170.78416675210622):(436.10953315937553, -612.4834982574257)
(83.86250291177613, -170.78416675210622):(389.6463996245507, -585.7319971313144)
(83.86250291177613, -170.78416675210622):(411.4699926484836, -627.9712094357005)
(83.86250291177613, -170.78416675210622):(386.83045213759164, -524.4851392899544)
(83.86250291177613, -170.78416675210622):(362.1909116266997, -539.2688635964896)
(83.86250291177613, -170.78416675210622):(337.55137111580774, -555.4605616465043)
(83.86250291177613, -170.78416675210622):(362.89489849843943, -597.6997739508905)
(83.86250291177613, -170.78416675210622):(313.61581747665554, -567.4283384660804)
(83.86250291177613, -170.78416675210622):(338.2553579875475, -614.5954588726449)
(83.86250291177613, -1

...

(582.8746397454761, -311.05478356541806):(232.0048472965808, -258.7060459398356)
(582.8746397454761, -311.05478356541806):(255.79193104942544, -245.1564412704937)
(582.8746397454761, -311.05478356541806):(281.084526432197, -229.8002226452396)
(582.8746397454761, -311.05478356541806):(304.8716101850416, -216.55172030188308)
(582.8746397454761, -311.05478356541806):(329.56200091584236, -202.39991098057044)
(582.8746397454761, -311.05478356541806):(357.2634149064969, -187.94699933327243)
(582.8746397454761, -311.05478356541806):(92.59447036535204, -338.7992646519456)
(582.8746397454761, -311.05478356541806):(370.15373505832883, -180.93277541647447)
(582.8746397454761, -311.05478356541806):(382.2545131726996, -172.7717855253872)
(582.8746397454761, -311.05478356541806):(113.67163318432833, -386.9756368096056)
(582.8746397454761, -311.05478356541806):(136.25430763323146, -374.3293391182198)
(582.8746397454761, -311.05478356541806):(160.94469836403223, -360.1775297969072)
(582.8746397454761,

(575.1626264439374, -317.0040509694623):(540.2995901768616, -455.49442585945695)
(575.1626264439374, -317.0040509694623):(536.0756689464229, -466.75821580729325)
(575.1626264439374, -317.0040509694623):(519.8839708964082, -477.3180188833898)
(575.1626264439374, -317.0040509694623):(479.7567192072413, -469.57416329425234)
(575.1626264439374, -317.0040509694623):(457.22913931156864, -481.54194011382845)
(575.1626264439374, -317.0040509694623):(506.50822033335254, -506.1814806247204)
(575.1626264439374, -317.0040509694623):(525.5158658703263, -490.69376944644546)
(575.1626264439374, -317.0040509694623):(499.4683516159548, -512.5173624703783)
(575.1626264439374, -317.0040509694623):(484.6846273094196, -525.893113033434)
(575.1626264439374, -317.0040509694623):(460.7490736702675, -538.5648767247499)
(575.1626264439374, -317.0040509694623):(434.701559415896, -494.91769067688404)
(575.1626264439374, -317.0040509694623):(410.7660057767438, -510.40540185515897)
(575.1626264439374, -317.00405096

(592.7900854188831, -315.9023347835282):(84.76580988973231, -226.26227031491163)
(592.7900854188831, -315.9023347835282):(108.55289364257695, -213.3148702975405)
(592.7900854188831, -315.9023347835282):(132.64107972140692, -200.066367954184)
(592.7900854188831, -315.9023347835282):(157.63257277819315, -186.44148770334562)
(592.7900854188831, -315.9023347835282):(184.43067979089153, -171.6874737300622)
(592.7900854188831, -315.9023347835282):(162.14520987595813, -128.05778036416322)
(592.7900854188831, -315.9023347835282):(208.51886586972154, -157.83676673473497)
(592.7900854188831, -315.9023347835282):(182.66247959549878, -106.25818128715127)
(592.7900854188831, -315.9023347835282):(232.30594962256617, -145.79267369531996)
(592.7900854188831, -315.9023347835282):(208.95023142366023, -102.41119321473741)
(592.7900854188831, -315.9023347835282):(190.3564557403265, -69.7117945992195)
(592.7900854188831, -315.9023347835282):(259.7062612872353, -131.33976204802195)
(592.7900854188831, -315.

Betweenness centrality:{(49.94139109722019, -164.604167052095): 0.0, (60.67762381090223, -185.23707839940425): 0.01588945471308655, (69.8174961380252, -145.36922669002564): 0.0, (83.86250291177613, -170.78416675210622): 0.020652560606614384, (35.68613075411612, -196.6789667868485): 0.0006493511364820535, (106.74627968666468, -157.2345620827643): 0.008590475971779978, (61.279828462873056, -239.51077265826814): 0.007387700410112457, (84.76580988973231, -226.26227031491163): 0.02518097046380786, (108.55289364257695, -213.3148702975405): 0.03955071755238593, (132.64107972140692, -200.066367954184): 0.04569146432317399, (157.63257277819315, -186.44148770334562): 0.036462380150065844, (184.43067979089153, -171.6874737300622): 0.044965835149635765, (162.14520987595813, -128.05778036416322): 0.0, (208.51886586972154, -157.83676673473497): 0.044554305780556205, (182.66247959549878, -106.25818128715127): 0.0, (232.30594962256617, -145.79267369531996): 0.04979991667964539, (208.95023142366023, -1

/home/maltaweel/.local/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
<ipython-input-1-280855836816>:190: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(path_output, driver="ESRI Shapefile")
